# Background of this notebook 

I used XGB classifier on this notebook: [Digits-Recognizer -Simple XGB Classifier](http://https://www.kaggle.com/satoshiss/digits-recognizer-simple-xgb-classifier).  Next I am going to use convolutional neural networks with help of "Deep Learning for Dummies". I am reading the book at the moment. Hope this project helps me to understand the topic better.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import keras
from keras.models import Sequential 
from keras.layers import Conv2D, AveragePooling2D
from keras.layers import Dense, Flatten
from keras.losses import categorical_crossentropy

In [ ]:
df_train = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
df_test = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")

In [ ]:
df_train

In [ ]:
df_train.label.unique()

In [ ]:
from matplotlib.pyplot import imshow

width=5
height=5
rows = 2
cols = 3
axes=[]

fig=plt.figure()
fig.set_size_inches(8,10)
for i in range(rows*cols):
    sample=np.reshape(df_train[df_train.columns[1:]].iloc[i].values/255,(28,28))
    axes.append(fig.add_subplot(rows,cols,i+1))
    plt.title("Labeled class : {}".format(df_train["label"].iloc[i]))
    plt.imshow(sample, 'gray')
fig.tight_layout()
plt.show()

# Explanatory Data Analysis

In [ ]:
plt.figure(figsize=(8,6))
ax = sns.countplot(x='label',data=df_train)

plt.title("Label Distribution")
total= len(df_train.label)
for p in ax.patches:
    percentage = f'{100 * p.get_height() / total:.1f}%\n'
    x = p.get_x() + p.get_width() / 2
    y = p.get_height()
    ax.annotate(percentage, (x, y), ha='center', va='center')

In [ ]:
# separate target values from df_train
targets = df_train.label
features = df_train.drop("label",axis=1)

In [ ]:
features 

# Preprocessing the Data

In [ ]:
features = features/255
df_test = df_test/255

In [ ]:
X_train,X_val, y_train,y_val = train_test_split(features,targets,test_size=0.2, random_state=1)

In [ ]:
from tensorflow.keras.utils import to_categorical

num_classes = len(np.unique(y_train))
print(y_train[0], end= ' => ')
y_train = to_categorical(y_train,10)
y_val =  to_categorical(y_val,10)
print(y_train[0])

In [ ]:
# 784 = 28*28
img_rows,img_cols = 28,28

X_train = X_train.values.reshape(len(X_train),img_rows,img_cols,1)
X_val = X_val.values.reshape(len(X_val),img_rows,img_cols,1)

df_test = df_test.values.reshape(len(df_test),img_rows,img_cols,1)

input_shape = (img_rows,img_cols,1)

# Making a Model and Predictions

In [ ]:
lenet = Sequential()

lenet.add(Conv2D(6,kernel_size=(5,5),activation='relu',input_shape=input_shape,padding='same',name ='C1'))
lenet.add(AveragePooling2D(pool_size= (2,2),strides=(1,1),padding='valid'))
lenet.add(Conv2D(120, kernel_size=(5,5),activation='relu',name='C2'))
lenet.add(Flatten())
lenet.add(Dense(84,activation='relu',name='C3'))
lenet.add(Dense(10,activation='softmax',name='Output'))

lenet.compile(loss=categorical_crossentropy,optimizer='SGD',metrics = ['accuracy'])

lenet.summary()


In [ ]:
batch_size=64
epochs = 50
history = lenet.fit(X_train,y_train, batch_size = batch_size,epochs = epochs, validation_data = (X_val,y_val))

In [ ]:
predictions = lenet.predict(df_test)
predictions = [np.argmax(a) for a in predictions]



In [ ]:
output = pd.read_csv("../input/digit-recognizer/sample_submission.csv")
output['Label'] = predictions
output.to_csv('submission.csv',index=False)
